In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as lcpinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
PINECONE_API_KEY = "d18ddb58-7c71-4208-8072-2dda5ed47ef4"
PINECONE_API_ENV = "aws-starter"

In [ ]:
def load_documents(datadir):
    loader = DirectoryLoader(datadir,glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [ ]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
extrated_docs = load_documents("C:\genaipjts\medicalchatbot\MedicalChatbot\data")

In [ ]:
extracted_data = load_pdf("C:/genaipjts/medicalchatbot/MedicalChatbot/data/")
extracted_data

In [ ]:
"""
def split_chunks(docList):
    print("inside split_chunks func : ",len(docList))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(docList)
    print("inside split_chunks func after chubking: ",len(text_chunks))
    return text_chunks
"""

#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

In [ ]:
def ret_huggingface_embeddings_obj():
    emdObj = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return emdObj

In [ ]:
embeddingObj = ret_huggingface_embeddings_obj()

In [ ]:
enbdQr = embeddingObj.embed_query("My name is Deepak Nema")
print(len(enbdQr))
enbdQr

#query_result = embeddingObj.embed_query("My name is Deepak Nema")
#print("Length", len(query_result))

In [ ]:
# not working

from pinecone import Pinecone as pcclass, ServerlessSpec

#api_key = os.environ.get("PINECONE_API_KEY")

# configure client
pc = pcclass(api_key=PINECONE_API_KEY)

index_name="medical-chatbot"

#pc.from_texts()

docsearch=lcpinecone.from_texts([t.page_content for t in text_chunks], embeddingObj, index_name=index_name)



In [ ]:
!pip install -qU \
  pinecone-client[grpc]==3.0.0 \
  pinecone-datasets==0.7.0 \
  langchain-pinecone==0.0.3 \
  langchain-openai==0.0.7 \
  langchain==0.1.9

In [ ]:
# working
# Need not run again. Index exists in pinecone

import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader


#os.environ['OPENAI_API_KEY'] = '<YOUR_OPENAI_API_KEY>'
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

index_name = "medical-chatbot"
 
# path to an example text file

vectorstore_from_texts = PineconeVectorStore.from_texts(
        [t.page_content for t in text_chunks],
        index_name=index_name,
        embedding=embeddingObj
    )

In [ ]:

index_name = "medical-chatbot"
vectorstore_from_texts=PineconeVectorStore.from_existing_index(index_name, embeddingObj)

query = "What are Allergies"

docs = vectorstore_from_texts.similarity_search(query=query)

#docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
print(PROMPT)
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="C:\genaipjts\medicalchatbot\MedicalChatbot\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})


In [ ]:
from langchain.chains import RetrievalQA
reteieverObj = vectorstore_from_texts.as_retriever(search_kwargs={'k': 2})
#reteieverObj = vectorstore_from_texts.as_retriever()
print(type(reteieverObj))

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=reteieverObj,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)


In [ ]:
# if required
!pip install -qU langchain
!pip install --upgrade pydantic

In [ ]:
#user_input=input(f"Input Prompt:")
user_input = "What are the colors of blood cells?"
result=qa({"query": user_input})
print("Response : ", result["result"])